## Notebook to convert audio files from the ESC-50 dataset into spectrograms using a variety of transforms

### Requires librosa for linear STFT / mel STFT / CQT and pyWavelets for CWT

In [1]:
%matplotlib inline
## kernel: python 3.5
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa # https://github.com/librosa/librosa
import librosa.display
import re
import warnings

import tiffspect


# Set some project parameters
K_SR = 22050 #sampling rate
K_FFTSIZE = 1024 # also used for window length where that parameter is called for
K_HOP = 512 #fft hop length
K_DUR = 5.0 # make all files this duration (sec)

main_dir = "./"

# location of subdirectories of ogg files organized by category
K_OGGDIR = main_dir + "toy_data"
# location to write the wav files (converted from ogg)
K_WAVEDIR = main_dir + "ESC-50-wav"

In [2]:
def get_subdirs(a_dir):
    """ Returns a list of sub directory names in a_dir """ 
    return [name for name in os.listdir(a_dir)
            if (os.path.isdir(os.path.join(a_dir, name)) and not (name.startswith('.')))]

def listDirectory(directory, fileExtList):                                        
    """Returns a list of file info objects in directory that extension in the list fileExtList - include the . in your extension string"""
    fnameList = [os.path.normcase(f)
                for f in os.listdir(directory)
                    if (not(f.startswith('.')))]            
    fileList = [os.path.join(directory, f) 
               for f in fnameList
                if os.path.splitext(f)[1] in fileExtList]  
    return fileList , fnameList

def dirs2labelfile(parentdir, labelfile):
    """takes subdirectories of parentdir and writes them, one per line, to labelfile"""
    namelist = get_subdirs(parentdir)
    with open(labelfile, mode='wt', encoding='utf-8') as myfile:
    #with open(labelfile, mode='wt') as myfile:
        myfile.write('\n'.join(namelist))

### Convert and standardize the .ogg files into .wav files

In [3]:
# Use this for .ogg to .wav conversion. Clip/pad to same length.

def esc50Ogg2Wav (topdir, outdir, dur, srate) :
    """ 
        Creates regularlized wave files for the ogg files in the ESC-50 dataset. 
        Creates class folders for the wav files in outdir with the same structure found in topdir.
        
        Parameters
            topdir - the ESC-50 dir containing class folders. 
            outdir - the top level directory to write wave files to (written in to class subfolders)
            dur - (in seconds) all files will be truncated or zeropadded to have this duration given the srate
            srate - input files will be resampled to srate as they are read in before being saved as wav files
    """ 
    sample_length = int(dur * srate)
    try:
        os.stat(outdir)  # test for existence
    except:
        os.mkdir(outdir) # create if necessary
        
    subdirs = get_subdirs(topdir)
    for subdir in subdirs :

        fullpaths, _ = listDirectory(topdir + '/' + subdir, '.ogg')

        for idx in range(len(fullpaths)) : 
            fname = os.path.basename(fullpaths[idx])
            # librosa.load resamples to sr, clips to duration, combines channels. 
            audiodata, samplerate = librosa.load(fullpaths[idx], sr=srate, mono=True, duration=dur) # resamples if necessary (some esc-50 files are in 48K)
            # just checking ..... 
            if (samplerate != srate) :
                print('You got a sound file ' + subdir  +  '/' +  fname + ' with sample rate ' + str(samplerate) + '!')
                print(' ********* BAD SAMPLE RATE ******** ')
            if (audiodata.ndim != 1) :
                print('You got a sound file ' + subdir  +  '/' +  fname + ' with ' + str(audiodata.ndim) + ' channels!')
                audiodata = stereo2mono(audiodata)
            if (len(audiodata) > sample_length) :
                print('You got a long sound file ' + subdir  +  '/' +  fname + ' with shape ' + str(audiodata.shape) + '!')
                audiodata = np.resize(audiodata, sample_length)
                # print('  ..... and len(audiodata) = ' + str(len(audiodata)) + ', while sample_length is sposed to be ' + str(sample_length))
                print('trimming data to shape ' + str(audiodata.shape))
            if (len(audiodata) < sample_length) :
                print('You got a short sound file ' + subdir  +  '/' +  fname + ' with shape ' + str(audiodata.shape) + '!')
                audiodata = np.concatenate([audiodata, np.zeros((sample_length-len(audiodata)))])
                print('zero padding data to shape ' + str(audiodata.shape))
            
            # write the file out as a wave file
            try:
                os.stat(outdir + '/'  + subdir) # test for existence
            except:
                os.mkdir(outdir + '/' + subdir) # create if necessary
            print('creating ' + outdir + '/'  + subdir)
            librosa.output.write_wav(outdir + '/' + subdir + '/' + os.path.splitext(fname)[0] + '.wav', audiodata, samplerate)
    print("COMPLETE")    

In [4]:
esc50Ogg2Wav(K_OGGDIR, K_WAVEDIR, K_DUR, K_SR)

creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/102 - Rooster
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
creating ./ESC-50-wav/101 - Dog
COMPLETE


### Ok now we will transform the wav files into spectrograms

In [5]:
# First choose the desired transform
transform = "stft" #choose the signal processing technique: ["stft","mel","cqt","cwt","mfcc"]

if transform == "stft":
    # location to write the linear spectrogram files (converted from wave files)
    K_SPECTDIR = main_dir + 'stft'
elif transform == "mel":
    # location to write the mel spec files (converted from wave files)
    K_SPECTDIR = main_dir + 'mel'
elif transform == "cqt":
    # location to write the CQT spec files (converted from wave files)
    K_SPECTDIR = main_dir + 'cqt'
elif transform == "cwt":
    # location to write the wavelet files (converted from wave files)
    K_SPECTDIR = main_dir + 'cwt'
elif transform == "mfcc":
    # location to write the mfcc files (converted from wave files)
    K_SPECTDIR = main_dir + 'mfcc'
else:
    raise ValueError("Transform not supported! Please choose from ['stft','mel','cqt','cwt','mfcc']")

In [6]:
#routines to convert wav to spectrograms using Librosa as backend. For wavelets we are using Pywavelet library.
import scipy
from PIL import TiffImagePlugin
from PIL import Image


def wav2stft(fname, srate, fftSize, fftHop, y_scale='linear', dur=None, showplt=False, dcbin=True) :
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = np.abs(librosa.stft(audiodata, n_fft=fftSize, hop_length=fftHop, win_length=fftSize,  center=False))

    if (dcbin ==  False) :
        S = np.delete(S, (0), axis=0)  # delete freq 0 row
            #note: a pure DC input signal bleeds into bin 1, too.

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis=y_scale, x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2cqt(fname, srate, fftSize, fftHop, dur=None, showplt=False) :
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.cqt(audiodata,hop_length=fftHop,n_bins=fftSize//2,bins_per_octave=128)

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='cqt_note', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2cwt(fname, srate, freq_bins, wavelet='morl', dur=None, showplt=False):
    import pywt
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    widths = np.arange(1, freq_bins+1) #no.of freq bins
    
    S, freqs = pywt.cwt(audiodata, widths, wavelet, sampling_period=1/srate)

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2mel(fname, srate, fftSize, fftHop, n_mels=128, dur=None, showplt=False):
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.feature.melspectrogram(audiodata, sr=srate, n_fft=fftSize, hop_length=fftHop, n_mels=n_mels)

    D = librosa.power_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='mel', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2mfcc(fname, srate, fftSize, fftHop, n_mfcc=128, dur=None, showplt=False):
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.feature.mfcc(audiodata,sr=srate, n_fft=fftSize, hop_length=fftHop, n_mfcc=n_mfcc)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return S  

In [7]:
def esc50get_fold(string):
    """get fold no. from ESC-50 dataset using the filename. Labels #1-5"""
    label_pos = string.index("-")
    return string[label_pos-1]

In [8]:
def wav2Spect(topdir, outdir, dur, srate, fftSize, fftHop, transform, neww=None, newh=None, newscale=None) :
    """ 
        Creates spectrograms for subfolder-labeled wavfiles. 
        Creates class folders for the spectrogram files in outdir with the same structure found in topdir.
        Outputs both tif files (greater resolution) and png files (for TFRecords)
        
        Parameters
            topdir - the dir containing class folders containing wav files. 
            outdir - the top level directory to write wave files to (written in to class subfolders)
            dur - (in seconds) all files will be truncated or zeropadded to have this duration given the srate
            srate - input files will be resampled to srate as they are read in before being saved as wav files
            transform - desired transform, defined at the top of this notebook
            
        Scaling Params
            neww - scale width of spectrogram (in pixels) to this value 
            newh - scale height of spectrogram (in pixels) to this value. maintains aspect ratio if only neww is specified
            *if both neww and newh is left unspecified no scaling is carried out
            newscale - normalize intensity values of spectrogram from about [-80,0]dB to a new range. default is [0,255])
    """ 
    if newscale is None:
        newscale = [0,255]
    if newscale[0] != 0 or newscale[1] != 255:
        warnings.warn("Scaling of intensity values outside unit8 range.Change newscale param to  [0,255] else PNG files will not be generated!")
    
    subdirs = get_subdirs(topdir)
    count = 0
    for subdir in subdirs:
    
        fullpaths, _ = listDirectory(topdir + '/' + subdir, '.wav') 
        
        for idx in range(len(fullpaths)) : 
            fname = os.path.basename(fullpaths[idx])
            fold = esc50get_fold(fname)
            
            if transform == "stft":
                D = wav2stft(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "mel":
                D = wav2mel(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "cqt":
                D = wav2cqt(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "cwt":
                D = wav2cwt(fullpaths[idx], srate, 64, 'morl')
            elif transform == "mfcc":
                D = wav2mfcc(fullpaths[idx], srate, fftSize, fftHop)
            
            try:
                os.stat(outdir + '_tif/' + fold + '/' + subdir) # test for existence
            except:
                os.makedirs(outdir + '_tif/' + fold + '/' + subdir) # create if necessary
            
            tiffspect.specScale(D, outdir+'_tif/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0],
                                neww, newh, newscale)
            
            if newscale[0] == 0 and newscale[1] == 255:
            # output png if scale is correct
                try:
                    os.stat(outdir + '_png/' + fold + '/' + subdir) # test for existence
                except:
                    os.makedirs(outdir + '_png/' + fold + '/' + subdir) # create if necessary
                
                tiffspect.tif2png(outdir+'_tif/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0],
                                    outdir+'_png/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0])
            
            print(str(count) + ': ' + subdir + '/' + os.path.splitext(fname)[0])
            count +=1
    print("COMPLETE")     
 

In [9]:
wav2Spect(K_WAVEDIR, K_SPECTDIR, K_DUR, K_SR, K_FFTSIZE, K_HOP, transform)

0: 102 - Rooster/2-65750-A
1: 102 - Rooster/3-107219-A
2: 102 - Rooster/4-164064-A
3: 102 - Rooster/5-194930-B
4: 102 - Rooster/1-26806-A
5: 102 - Rooster/3-116135-A
6: 102 - Rooster/5-194930-A
7: 102 - Rooster/1-27724-A
8: 102 - Rooster/2-71162-A
9: 102 - Rooster/4-164021-A
10: 101 - Dog/3-136288-A
11: 101 - Dog/3-144028-A
12: 101 - Dog/2-114587-A
13: 101 - Dog/5-203128-A
14: 101 - Dog/1-30226-A
15: 101 - Dog/4-182395-A
16: 101 - Dog/2-114280-A
17: 101 - Dog/1-30344-A
18: 101 - Dog/5-9032-A
19: 101 - Dog/4-183992-A
COMPLETE


In [15]:
# output human readable class labels into text file (required for TFRecords)
dirs2labelfile(K_SPECTDIR+'_png/1', main_dir+'/labels.txt')

### We are done...alternative CQT transform below, which reads in a linear STFT spectrogram, undoes the intensity normalization, then does the conversion

In [11]:
# alternative CQT transform below, which converts from a linear spectrogram
# from lonce wyse: https://github.com/lonce/audioST/blob/master/dataPrep/ESC50_Convert.ipynb

K_QSPECTDIR = main_dir + 'esc50Qspect' #directory to output files

def logfmap(I, L, H) :
    """
    % [M,N] = logfmap(I,L,H)
    I - number of rows in the original spectrogram
    L - low bin to preserve
    H - high bin to preserver
    
    %     Return a maxtrix for premultiplying spectrograms to map
    %     the rows into a log frequency space.
    %     Output map covers bins L to H of input
    %     L must be larger than 1, since the lowest bin of the FFT
    %     (corresponding to 0 Hz) cannot be represented on a 
    %     log frequency axis.  Including bins close to 1 makes 
    %     the number of output rows exponentially larger.
    %     N returns the recovery matrix such that N*M is approximately I
    %     (for dimensions L to H).
    %     
    % Ported from MATLAB code written by Dan Ellis:
    % 2004-05-21 dpwe@ee.columbia.edu
    """
    ratio = (H-1)/H;
    opr = np.int(np.round(np.log(L/H)/np.log(ratio))) #number of frequency bins in log rep + 1
    print('opr is ' + str(opr))
    ibin = L*np.exp(list(range(0,opr)*-np.log(ratio)))  #fractional bin numbers (len(ibin) = opr-1)
    
    M = np.zeros((opr,I))
    eps=np.finfo(float).eps
    
    for i in range(0, opr) :
        # Where do we sample this output bin?
        # Idea is to make them 1:1 at top, and progressively denser below
        # i.e. i = max -> bin = topbin, i = max-1 -> bin = topbin-1, 
        # but general form is bin = A exp (i/B)

        tt = np.multiply(np.pi, (list(range(0,I))-ibin[i]))
        M[i,:] = np.divide((np.sin(tt)+eps) , (tt+eps));

    # Normalize rows, but only if they are boosted by the operation
    G = np.ones((I));
    print ('H is ' + str(H))
    G[0:H] = np.divide(list(range(0,H)), H)
    
    N = np.transpose(np.multiply(M,np.matlib.repmat(G,opr,1)))
                   
    return M,N

In [12]:
def esc50Spect2logFreqSpect(topdir, outdir, srate, fftSize, fftHop, lowRow=1, neww=None, newh=None, newscale=None) :
    """ 
        Creates psuedo constant-Q spectrograms from linear frequency spectrograms. 
        Creates class foldersfna in outdir with the same structure found in topdir.
        
        Parameters
            topdir - the dir containing class folders containing tif (log magnitude) spectrogram files. 
            outdir - the top level directory to write psuedo constantQ files to (written in to class subfolders)
            lowRow is the lowest row in the FFT that you want to include in the psuedo constant Q spectrogram
    """ 
    
    # First lets get the logf map we want
    LIN_FREQ_BINS = int(fftSize/2+1) #number of bins in original linear frequency mag spectrogram
    LOW_ROW = lowRow
    LOG_FREQ_BINS = int(fftSize/2+1) #resample the lgfmapped psuedo consantQ matrix to have this many frequency bins
    M,N = logfmap(LIN_FREQ_BINS,LOW_ROW,LOG_FREQ_BINS);
    
    if newscale is None:
        newscale = [0,255]
    if newscale[0] != 0 or newscale[1] != 255:
        warnings.warn("Scaling of intensity values outside unit8 range.Change newscale param to  [0,255] else PNG files will not be generated!")
    
    folds = get_subdirs(topdir)
    count = 0
    for fold in folds:
        subdir = get_subdirs(topdir + '/' + fold)
        for classes in subdir:
            try:
                os.stat(outdir + '_tif/' + fold + '/' + classes) # test for existence
            except:
                os.makedirs(outdir + '_tif/' + fold + '/' + classes) # create if necessary
            fullpaths, _ = listDirectory(topdir + '/' + fold + '/' + classes, '.tif')

            for idx in range(len(fullpaths)) : 
                fname = os.path.basename(fullpaths[idx])
                D, tiffinfo = tiffspect.invSpecScale(fullpaths[idx])            
                #print('tiffinfo is ' + str(tiffinfo))

                # Here's the beef
                MD = np.dot(M,D);
                MD = scipy.signal.resample(MD, LIN_FREQ_BINS) #downsample to something reasonable 

                #save
                tiffinfo["linFreqBins"] =  LIN_FREQ_BINS
                tiffinfo["lowRow"] = LOW_ROW
                tiffinfo["logFreqBins"] = LOG_FREQ_BINS
                tiffspect.specScale(MD, outdir+'_tif/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0],
                                    neww, newh, newscale, lwinfo=tiffinfo)
                
                if newscale[0] == 0 and newscale[1] == 255:
                # output png if scale is correct
                    try:
                        os.stat(outdir + '_png/' + fold + '/' + classes) # test for existence
                    except:
                        os.makedirs(outdir + '_png/' + fold + '/' + classes) # create if necessary

                    tiffspect.tif2png(outdir+'_tif/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0],
                                        outdir+'_png/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0])                
                
                print(str(count) + ': ' + classes + '/' + os.path.splitext(fname)[0])
                count +=1
    print("COMPLETE") 

In [13]:
esc50Spect2logFreqSpect(K_SPECTDIR+'_tif', K_QSPECTDIR, K_SR, K_FFTSIZE, K_HOP, lowRow=10)

opr is 2018
H is 513
0: 102 - Rooster/2-65750-A
1: 102 - Rooster/2-71162-A
2: 101 - Dog/2-114587-A
3: 101 - Dog/2-114280-A
4: 102 - Rooster/4-164064-A
5: 102 - Rooster/4-164021-A
6: 101 - Dog/4-182395-A
7: 101 - Dog/4-183992-A
8: 102 - Rooster/5-194930-A
9: 102 - Rooster/5-194930-B
10: 101 - Dog/5-203128-A
11: 101 - Dog/5-9032-A
12: 102 - Rooster/3-116135-A
13: 102 - Rooster/3-107219-A
14: 101 - Dog/3-144028-A
15: 101 - Dog/3-136288-A
16: 102 - Rooster/1-26806-A
17: 102 - Rooster/1-27724-A
18: 101 - Dog/1-30344-A
19: 101 - Dog/1-30226-A
COMPLETE


In [14]:
## example of using librosa spectrogram viewer as well as tiff file writer and reader
# import scipy
# %matplotlib inline

# # Just run this on any wav file in your system.
# audiodata, samplerate = librosa.load('esc50Wave/109 - Sheep/4-196671-B.wav', sr=K_SR, mono=True, duration=K_DUR) # resamples if necessary (some esc-50 files are in 48K)
# #audiodata, samplerate = librosa.load(K_WAVEDIR + '/1/BeingRural_short.wav', sr=K_SR, mono=True, duration=K_DUR) # resamples if necessary (some esc-50 files are in 48K)

# print('audiodata max is ' + str(np.max(audiodata)) + ', and audiodata sum is ' + str(np.sum(audiodata)))

# # compute spectrogram and display
# S = np.abs(librosa.stft(audiodata, n_fft=K_FFTSIZE, hop_length=K_HOP, win_length=K_FFTSIZE,  center=False))
# print('esc50Wav2Spect" magspec max is ' + str(np.max(S)) +  ', and magspec sum is ' + str(np.sum(S)) + ', and magspec min is ' + str(np.min(S)))

# Sfoo = np.delete(S, (0), axis=0)  # delete freq 0 row
# print('esc50Wav2Spect" Sfoo max is ' + str(np.max(Sfoo)) +  ', and Sfoo sum is ' + str(np.sum(Sfoo)) + ', and Sfoo min is ' + str(np.min(Sfoo)))


# #D = librosa.logamplitude(S**2, ref_power=np.max)
# # computes amplitude_to_dB (which is in the man, but not in the library for some reason)
# D = librosa.amplitude_to_db(S, ref=np.max)
# print('spectrogram D shape is ' + str(D.shape))
# print('esc50Wav2Spect" ampDB max is ' + str(np.max(D)) +  ', and ampDB min is ' + str(np.min(D)))


# plt.subplot(3, 1, 1)
# librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=K_SR, hop_length=K_HOP)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Linear-frequency magnitude spectrogram')

# # save spectrogram somewhere
# tiffspect.logSpect2Tiff(D, 'foo.tif')

# # ====================
# # Proof that spectrogram writer/reader is working properly

# D1 = tiffspect.Tiff2LogSpect('foo.tif')
# plt.subplot(3, 1, 3)
# librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=K_SR, hop_length=K_HOP)
# plt.colorbar(format='%+2.0f dB')
# plt.title('From Tiff: Linear-frequency magnitude spectrogram')
# tiffspect.logSpect2Tiff(D, 'foo.tif')